<a href="https://colab.research.google.com/github/AlexKitipov/FinRL/blob/master/FractalNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy
import cartopy

In [ ]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [ ]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
!pip install matplotlib-venn

In [ ]:
from fractalnet.blocks import FractalBlock


In [ ]:
# Създай работна папка
!mkdir -p /content/FractalNet/fractalnet

# Премини в нея
%cd /content/FractalNet

/content/FractalNet


In [ ]:
%run setup.py install

INFO:root:running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

       

In [ ]:
%%bash
set -e

# 1. Отиваме в работната папка (ако няма – се създава)
mkdir -p /content/FractalNet
cd /content/FractalNet

# 2. Пакетната конфигурация
cat << 'EOF' > setup.py
from setuptools import setup, find_packages

setup(
    name='fractalnet',
    version='0.1.0',
    description='Fractal Neural Network blocks for PyTorch',
    author='Твоето Име',
    packages=find_packages(),
    install_requires=['torch>=1.7.0'],
    python_requires='>=3.7'
)
EOF

# 3. Създаваме пакета и кода
mkdir -p fractalnet

cat << 'EOF' > fractalnet/blocks.py
import torch, random
import torch.nn as nn

class FractalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, depth, drop_rate=0.15):
        super().__init__()
        self.depth     = depth
        self.drop_rate = drop_rate

        if depth == 1:
            self.block = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)
            )
        else:
            self.left  = FractalBlock(in_ch, out_ch, depth-1, drop_rate)
            self.right = FractalBlock(in_ch, out_ch, depth-1, drop_rate)

    def forward(self, x):
        if self.depth == 1:
            return self.block(x)

        # drop-path
        if self.training and random.random() < self.drop_rate:
            return self.left(x) if random.random()<0.5 else self.right(x)

        out_l = self.left(x)
        out_r = self.right(x)
        return 0.5 * (out_l + out_r)
EOF

# празен __init__.py
touch fractalnet/__init__.py

# 4. Инсталираме локално
pip install -e .


Obtaining file:///content/FractalNet
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: fractalnet
    Found existing installation: fractalnet 0.1.0
    Uninstalling fractalnet-0.1.0:
      Successfully uninstalled fractalnet-0.1.0
  Running setup.py develop for fractalnet


DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/fractalnet-0.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import torch
from fractalnet.blocks import FractalBlock

# Пример за тестване на FractalBlock
block = FractalBlock(in_ch=3, out_ch=16, depth=3)
x     = torch.randn(1, 3, 32, 32)
out   = block(x)
print("Фрактален блок:", block)
print("Форма на изхода:", out.shape)


Фрактален блок: FractalBlock(
  (left): FractalBlock(
    (left): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (right): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (right): FractalBlock(
    (left): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (right): FractalBlock(
      (block): Sequential(
        (0): Con

In [ ]:
%%bash
cat > fractalnet/model.py << 'EOF'
import torch.nn as nn
from fractalnet.blocks import FractalBlock

class FractalNet(nn.Module):
    """
    Прост примерен FractalNet за CIFAR-10:
     - Stem: Conv(3→64)
     - FractalBlock: 64→128
     - Global avg-pool + FC(128→num_classes)
    """
    def __init__(self, num_classes=10, depth=3, drop_rate=0.15):
        super().__init__()
        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        # Един единичен фрактален блок
        self.fractal = FractalBlock(64, 128, depth, drop_rate)
        # Класификатор
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.fractal(x)
        x = self.global_pool(x).view(x.size(0), -1)
        return self.fc(x)
EOF


In [ ]:
!touch fractalnet/__init__.py

In [ ]:
pip install -e .


Obtaining file:///content/FractalNet
  Preparing metadata (setup.py) ... done
  Attempting uninstall: fractalnet
    Found existing installation: fractalnet 0.1.0
    Uninstalling fractalnet-0.1.0:
      Successfully uninstalled fractalnet-0.1.0
  Running setup.py develop for fractalnet


In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from fractalnet.model import FractalNet

# 1. Настройка на данните
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,)*3, (0.5,)*3)
])

train_ds = datasets.CIFAR10(root='.', train=True,  download=True, transform=transform)
test_ds  = datasets.CIFAR10(root='.', train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True,  num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False, num_workers=2)

# 2. Модел, loss, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = FractalNet(num_classes=10, depth=3, drop_rate=0.15).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# 3. Един епох тренировка
model.train()
running_loss = 0.0
for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if (i+1) % 100 == 0:
        print(f"[Batch {i+1}/{len(train_loader)}] Loss: {running_loss/100:.4f}")
        running_loss = 0.0

# 4. Бърз тест за accuracy
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images).argmax(dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

print(f"Test Accuracy: {100*correct/total:.2f}%")


[Batch 100/391] Loss: 2.0097
[Batch 200/391] Loss: 1.8289


In [ ]:
import torch
from fractalnet.blocks import FractalBlock

# Създай един фрактален блок
block = FractalBlock(in_ch=3, out_ch=16, depth=3)
print(block)

# Пробно подаване на един “dummy” тензор
x = torch.randn(1, 3, 32, 32)
out = block(x)
print("Output shape:", out.shape)


FractalBlock(
  (left): FractalBlock(
    (left): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (right): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (right): FractalBlock(
    (left): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (right): FractalBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kerne